# HST QSO Dataset used by Neeleman and Ribaduo

In [24]:
# imports
import glob, os
import pdb
from shutil import copy

from astropy.table import Table

from linetools import utils as ltu

## Summary file

In [22]:
dat_dir = summ_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/'

In [17]:
summ_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/hstqso.lst'

In [18]:
hstqso = Table.read(summ_file, format='ascii')

In [6]:
hstqso[0:3]

QSO_NAME,QSO_ALT_NAME,RA,DEC,QSO_ZEM,INST,GRATE,PID,SPEC_FILE
str10,str24,float64,float64,float64,str4,str17,str19,str23
J0000-1245,PHL2525,0.1017646,-12.76326,0.2,COS,G130M-G160M,12604,J0000-1245_COS.fits
J0001+0709,0,0.4190833,7.1651722,3.234,STIS,G230L,8569,J0001+0709_STIS.fits
J0004-4157,B0002-4214,1.2,-41.96,2.76,FOS,G270H,6577,B0002-4214_FOS-H.fits


## DATE-OBS

In [10]:
date_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/date_obs_fosh.txt'

In [11]:
date_fosh = Table.read(date_file, format='ascii')

In [12]:
date_fosh[0:3]

SPEC,GRATING,DATE-OBS
str18,str5,str10
B0002-4214_FOS-H,G270H,1996-6-2
B0002+0507_FOS-H,G270H,1993-11-4
B0002+0507_FOS-H,G270H,1993-11-4


## RA/DEC

In [16]:
radec_file = os.getenv('RAW_IGMSPEC')+'/HSTQSO/all_qso_table.txt'

In [17]:
radec = Table.read(radec_file, format='ascii')

In [18]:
radec[0:3]

File_ID,RA,DEC,GL,GB,ZEM,Zlls,NHI,NHI+,NHI-,LLS_TYPE
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9
B0002-4214,1.20083332,-41.95777893,332.680756,-72.370941,2.76,2.301,17.81,0.0,0.0,"R1,R2"
B0002+0507,1.33416665,5.40311098,102.310738,-55.611916,1.9,0.0,0.0,0.0,0.0,none
B0003+0146,1.448125,2.05061102,100.481804,-58.817158,0.234,0.0,0.0,0.0,0.0,none


In [19]:
fl = [row['File_ID'][0] for row in radec]

### Worry about lenses

In [3]:
q1318 = ltu.radec_to_coord(('13:18:54.67','+29:03:01.49'))

In [5]:
q1318a = ltu.radec_to_coord(('13:18:53.58','+29:03:30.52'))

In [7]:
q1318.separation(q1318a).to('arcsec')

<Angle 32.3576307359541 arcsec>

## Bad spectra

In [9]:
bad1 = '/raid/Raw_IGMspec//HSTQSO/J1330-2056_STIS.fits.gz'

In [10]:
hdu1 = fits.open(bad1)
hdu1.info()

Filename: /raid/Raw_IGMspec//HSTQSO/J1330-2056_STIS.fits.gz
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       4   ()              
1                BinTableHDU     43   1R x 3C      [1024D, 1024D, 1024D]   


In [11]:
data1 = Table(hdu1[1].data)
data1

WAVE [1024],FLUX [1024],ERR [1024]
float64,float64,float64
1118.6217041 .. 1715.77966309,0.0 .. 0.0,-0.0 .. -0.0


In [12]:
np.max(data1['ERR'])

-0.0

### Pushing these into a BAD folder

## Grabbing new COS files

In [19]:
legacy_dir = os.getenv('DROPBOX_DIR')+'COS-CGM-Legacy/Analysis'

In [26]:
for row in hstqso:
    if row['INST'] == 'COS':
        new_file = legacy_dir + '/{:s}/{:s}_hsla.fits'.format(row['QSO_ALT_NAME'], row['QSO_ALT_NAME'])
        if not os.path.isfile(new_file):
            pdb.set_trace()
        print('Copying {:s}'.format(new_file))
        copy(new_file,dat_dir)

Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/PHL2525/PHL2525_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/RXS-J00057-5007/RXS-J00057-5007_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/PG0003+158/PG0003+158_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/MRK335/MRK335_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/QSO-B0007+107/QSO-B0007+107_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/SDSSJ001224.01-102226.5/SDSSJ001224.01-102226.5_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/QSO-B0026+129/QSO-B0026+129_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/HS-0033+4300/HS-0033+4300_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/SDSSJ004222.29-103743.8/SDSSJ004222.29-103743.8_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/PG0044+030/PG0044+030_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/Analysis/IO-AND/IO-AND_hsla.fits
Copying /u/xavier/Dropbox/COS-CGM-Legacy/A

### Then I gzipped them

## Test

In [27]:
from igmspec.igmspec import IgmSpec

In [28]:
igmsp = IgmSpec(db_file='/Users/xavier/local/Python/igmspec/igmspec/../DB/IGMspec_DB_v02.hdf5')

Using /Users/xavier/local/Python/igmspec/igmspec/../DB/IGMspec_DB_v02.hdf5 for the catalog file
Using /Users/xavier/local/Python/igmspec/igmspec/../DB/IGMspec_DB_v02.hdf5 for the DB file
Available surveys: [u'HSTQSO', u'2QZ', u'ESI_DLA', u'HDLA100', u'COS-Halos', u'HST_z2', u'COS-Dwarfs', u'XQ-100']


In [29]:
spec, meta = igmsp.spec_from_coord((1.4968168,16.163614))

Your search yielded 1 match[es]
Staged 0 spectra totalling 0 Gb
No spectra matching in survey 2QZ
Staged 0 spectra totalling 0 Gb
No spectra matching in survey COS-Dwarfs
Staged 0 spectra totalling 0 Gb
No spectra matching in survey COS-Halos
Staged 0 spectra totalling 0 Gb
No spectra matching in survey ESI_DLA
Staged 0 spectra totalling 0 Gb
No spectra matching in survey HDLA100
Staged 2 spectra totalling 0.001952 Gb
Loaded spectra
Staged 0 spectra totalling 0 Gb
No spectra matching in survey HST_z2
Staged 0 spectra totalling 0 Gb
No spectra matching in survey XQ-100


/Users/xavier/local/Python/linetools/linetools/spectra/xspectrum1d.py:295: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [30]:
meta

[<Table length=2>
  QSO_NAME  QSO_ALT_NAME     RA        DEC     ... SURVEY_ID  DATE-OBS  TELESCOPE
   str10       str24      float64    float64   ...   int64     str10       str3  
 ---------- ------------ --------- ----------- ... --------- ---------- ---------
 J0005+1609   PG0003+158 1.4968168   16.163614 ...         6 2011-10-22       HST
 J0006+1609   B0003+1553   1.49675 16.16363907 ...         7 1993-11-07       HST]

In [32]:
meta[0][['QSO_NAME', 'INSTR']]

QSO_NAME,INSTR
str10,str4
J0005+1609,COS
J0006+1609,FOS
